In [1]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.naive_bayes
import sklearn.ensemble as ske
import sklearn.neighbors
import sklearn.pipeline
import sklearn.linear_model
import joblib
from sklearn import preprocessing
from sklearn import tree, linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel
import sys
import sqlite3
import importlib
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn import metrics

In [2]:
themida = pd.read_pickle('/kaggle/input/packer-balance-pickle/themida_balance.pickle')

In [3]:
themida = themida.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
themida = themida.drop(['benign_vt','malicious_vt'], axis=1)

In [4]:
string_columns = themida.select_dtypes(include=['object']).columns
themida = themida.drop(string_columns, axis=1)

In [5]:
themida.head()

,malicious,packed,imp_setupremovefilelogentrya,imp_vardeccmp,imp_setdlgitemint,imp_?append@?$basic_string@du?$char_traits@d@std@@v?$allocator@d@2@@std@@qaeaav12@abv12@ii@z,imp_zwwritefile,imp_glstencilop,imp_comparelinkinforeferents,imp__setmbcp,...,opcode_xor sub mov,opcode_xor sub mov add,opcode_xor test,opcode_xor test je,opcode_xor xor,opcode_xor xor mov,opcode_xor xor pop,opcode_xor xor pop mov,opcode_xor xor xor,opcode_xor xor xor pop
0,True,True,False,False,False,False,False,False,False,False,...,0.001402,0.001515,0.0,0.0,0.001377,0.000000,0.001793,0.000000,0.000990,0.001913
11,True,True,False,False,False,False,False,False,False,False,...,0.001331,0.001439,0.0,0.0,0.001307,0.000000,0.002882,0.000000,0.000000,0.000000
12,True,True,False,False,False,False,False,False,False,False,...,0.001426,0.001542,0.0,0.0,0.002309,0.001142,0.003828,0.002166,0.001706,0.003295
13,True,True,False,False,False,False,False,False,False,False,...,0.001484,0.001604,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
14,False,True,False,False,False,False,False,False,False,False,...,0.001513,0.001636,0.0,0.0,0.001486,0.000000,0.001936,0.002298,0.001069,0.002065


In [6]:
df_7000 = themida.sample(n=7000, random_state=102)
df_3000 = themida.drop(df_7000.index)

In [7]:
data = df_7000.sample(n=3500, random_state=102)

In [8]:
#Kiểm tra số lượng mẫu
PM = data[(data['malicious'] == True) & (data['packed'] == True)].shape[0]
PB = data[(data['malicious'] == False) & (data['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

Packed Malware: 2241 samples
Packed Benign: 1259 samples
Total TRAIN set: 3500 samples


In [9]:
#Chia tỉ lệ 50 - 50
data_PM = data[(data['malicious'] == True) & (data['packed'] == True)].sample(n=(PM-PB), random_state=102)
data = data.drop(data_PM.index)

In [10]:
#Số lượng tập train
PM = data[(data['malicious'] == True) & (data['packed'] == True)].shape[0]
PB = data[(data['malicious'] == False) & (data['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

Packed Malware: 1259 samples
Packed Benign: 1259 samples
Total TRAIN set: 2518 samples


In [11]:
#Xử lý tập train
y = data['malicious'].values
data = data.drop(['malicious','packed'], axis=1)

In [12]:
PM = df_3000[(df_3000['malicious'] == True) & (df_3000['packed'] == True)].shape[0]
PB = df_3000[(df_3000['malicious'] == False) & (df_3000['packed'] == True)].shape[0]

df_3000_PM = df_3000[(df_3000['malicious'] == True) & (df_3000['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000 = df_3000.drop(df_3000_PM.index)

PM = df_3000[(df_3000['malicious'] == True) & (df_3000['packed'] == True)].shape[0]
PB = df_3000[(df_3000['malicious'] == False) & (df_3000['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 1089 samples
Packed Benign: 1089 samples
Total TEST set: 2178 samples


In [13]:
# model = SelectFromModel(fsel, prefit=True)

X_train = data
X_test = df_3000.drop(['malicious','packed'], axis=1)

data = data.loc[:, (data != 0).any(axis=0)]
x_train = data
X_test = df_3000.drop(['malicious','packed'], axis=1)
y_train = y
y_test = df_3000['malicious'].values

In [14]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train, y_train)
random_forest_preds = random_forest.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test))

The accuracy of the Random Forests model is :	 0.8953168044077136


In [15]:
feature_scores_rf = pd.Series(random_forest.feature_importances_, index=X_train.columns).sort_values(ascending=False)

feature_scores_rf

ngram_b'\x03\x01\x00\x01\xa3\x82'    0.016312
ngram_b'\x0c\x06\n+\x06\x01'         0.012331
ngram_b'http:/'                      0.009091
ngram_b'\x01\x05\x00\x04\x82\x01'    0.008413
ngram_b'+\x06\x01\x05\x05\x07'       0.008289
                                       ...   
imp_pyexc_overflowerror              0.000000
imp_pydict_setitem                   0.000000
imp_pyexc_attributeerror             0.000000
imp__py_zerostruct                   0.000000
string_b'6!7(7E7l7'                  0.000000
Length: 56524, dtype: float64

In [16]:
top50 = pd.Series(random_forest.feature_importances_, index=X_train.columns).sort_values(ascending=False).head(50)

In [17]:
# Tạo DataFrame từ feature_scores_rf
df_features = pd.DataFrame(top50, columns=['Score'])

count_ngram = df_features[df_features.index.str.startswith('ngram_')].shape[0]
count_opcode = df_features[df_features.index.str.startswith('opcode_')].shape[0]
count_generic = df_features[df_features.index.str.startswith('generic_')].shape[0]
count_string = df_features[df_features.index.str.startswith('string_')].shape[0]
count_rich = df_features[df_features.index.str.startswith('rich_')].shape[0]
# count_api = df_features[df_features.index.str.startswith('imp_') or 'api_import_nb' == c].shape[0]
# count_dll = df_features[df_features.index.str.startswith('.dll') or 'dll_import_nb' == c].shape[0]
count_api = df_features[df_features.index.str.startswith('imp_') | (df_features.index == 'api_import_nb')].shape[0]
count_dll = df_features[df_features.index.str.endswith('.dll') | (df_features.index == 'dll_import_nb')].shape[0]
count_section = df_features[df_features.index.str.startswith('pesection')].shape[0]
count_header = df_features[df_features.index.str.startswith('header_')].shape[0]

print(f"'ngram': {count_ngram}")
print(f"'opcode ngram': {count_opcode}")
print(f"'generic': {count_generic}")
print(f"'strings': {count_string}")
print(f"'rich header': {count_rich}")
print(f"'api import': {count_api}")
print(f"'dll': {count_dll}")
print(f"'pe section': {count_section}")
print(f"'pe header': {count_header}")

'ngram': 50
'opcode ngram': 0
'generic': 0
'strings': 0
'rich header': 0
'api import': 0
'dll': 0
'pe section': 0
'pe header': 0


In [18]:
cols = list(themida.columns)

# **ngram features**

In [19]:
themida = pd.read_pickle('/kaggle/input/packer-balance-pickle/themida_balance.pickle')
themida = themida.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
themida = themida.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = themida.select_dtypes(include=['object']).columns
themida = themida.drop(string_columns, axis=1)

In [20]:
def remove_cols(df, ll):
    print("removing {} features".format(len(ll)))
    df.drop(columns=ll, axis=1, inplace=True)
    return df

In [21]:
# ngram = remove_cols(df, [c for c in cols if c.startswith('ngram_')])
ngram = remove_cols(themida, [c for c in cols if c.startswith('opcode_')])
ngram = remove_cols(themida, [c for c in cols if c.startswith('generic_')])
ngram = remove_cols(themida, [c for c in cols if c.startswith('string_')])
ngram = remove_cols(themida, [c for c in cols if c.startswith('rich_')])
ngram = remove_cols(themida, [c for c in cols if c.startswith('imp_') or 'api_import_nb' == c])
ngram = remove_cols(themida, [c for c in cols if c.endswith('.dll') or 'dll_import_nb' == c])
ngram = remove_cols(themida, [c for c in cols if c.startswith('pesection')])
ngram = remove_cols(themida, [c for c in cols if c.startswith('header_')])

removing 2500 features
removing 2 features
removing 16900 features
removing 66 features
removing 19169 features
removing 4306 features
removing 550 features
removing 29 features


In [22]:
df_7000_ngram = ngram.sample(n=7000, random_state=102)
df_3000_ngram = ngram.drop(df_7000.index)
data_ngram = df_7000_ngram.sample(n=3500, random_state=102)

In [23]:
#Kiểm tra số lượng mẫu
PM = data_ngram[(data_ngram['malicious'] == True) & (data_ngram['packed'] == True)].shape[0]
PB = data_ngram[(data_ngram['malicious'] == False) & (data_ngram['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_ngram = data_ngram[(data_ngram['malicious'] == True) & (data_ngram['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_ngram = data_ngram.drop(data_PM_ngram.index)

#Số lượng tập train
PM = data_ngram[(data_ngram['malicious'] == True) & (data_ngram['packed'] == True)].shape[0]
PB = data_ngram[(data_ngram['malicious'] == False) & (data_ngram['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y = data_ngram['malicious'].values
data_ngram = data_ngram.drop(['malicious','packed'], axis=1)

PM = df_3000_ngram[(df_3000_ngram['malicious'] == True) & (df_3000_ngram['packed'] == True)].shape[0]
PB = df_3000_ngram[(df_3000_ngram['malicious'] == False) & (df_3000_ngram['packed'] == True)].shape[0]

df_3000_PM_ngram = df_3000_ngram[(df_3000_ngram['malicious'] == True) & (df_3000_ngram['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_ngram = df_3000_ngram.drop(df_3000_PM_ngram.index)

PM = df_3000_ngram[(df_3000_ngram['malicious'] == True) & (df_3000_ngram['packed'] == True)].shape[0]
PB = df_3000_ngram[(df_3000_ngram['malicious'] == False) & (df_3000_ngram['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2241 samples
Packed Benign: 1259 samples
Total TRAIN set: 3500 samples
Packed Malware: 1259 samples
Packed Benign: 1259 samples
Total TRAIN set: 2518 samples
Packed Malware: 1089 samples
Packed Benign: 1089 samples
Total TEST set: 2178 samples


In [24]:
# model = SelectFromModel(fsel, prefit=True)

X_train_ngram = data_ngram
X_test_ngram = df_3000_ngram.drop(['malicious','packed'], axis=1)

data_ngram = data_ngram.loc[:, (data_ngram != 0).any(axis=0)]
x_train_ngram = data_ngram
X_test_ngram = df_3000_ngram.drop(['malicious','packed'], axis=1)
y_train_ngram = y
y_test_ngram = df_3000_ngram['malicious'].values

In [25]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train_ngram, y_train_ngram)
random_forest_preds = random_forest.predict(X_test_ngram)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_ngram))

The accuracy of the Random Forests model is :	 0.9288337924701561


# **opcode features**

In [26]:
themida = pd.read_pickle('/kaggle/input/packer-balance-pickle/themida_balance.pickle')
themida = themida.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
themida = themida.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = themida.select_dtypes(include=['object']).columns
themida = themida.drop(string_columns, axis=1)

In [27]:
opcode = remove_cols(themida, [c for c in cols if c.startswith('ngram_')])
# opcode = remove_cols(themida, [c for c in cols if c.startswith('opcode_')])
opcode = remove_cols(themida, [c for c in cols if c.startswith('generic_')])
opcode = remove_cols(themida, [c for c in cols if c.startswith('string_')])
opcode = remove_cols(themida, [c for c in cols if c.startswith('rich_')])
opcode = remove_cols(themida, [c for c in cols if c.startswith('imp_') or 'api_import_nb' == c])
opcode = remove_cols(themida, [c for c in cols if c.endswith('.dll') or 'dll_import_nb' == c])
opcode = remove_cols(themida, [c for c in cols if c.startswith('pesection')])
opcode = remove_cols(themida, [c for c in cols if c.startswith('header_')])

removing 13000 features
removing 2 features
removing 16900 features
removing 66 features
removing 19169 features
removing 4306 features
removing 550 features
removing 29 features


In [28]:
df_7000_opcode = opcode.sample(n=7000, random_state=102)
df_3000_opcode = opcode.drop(df_7000.index)
data_opcode = df_7000_opcode.sample(n=3500, random_state=102)

In [29]:
#Kiểm tra số lượng mẫu
PM = data_opcode[(data_opcode['malicious'] == True) & (data_opcode['packed'] == True)].shape[0]
PB = data_opcode[(data_opcode['malicious'] == False) & (data_opcode['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_opcode = data_opcode[(data_opcode['malicious'] == True) & (data_opcode['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_opcode = data_opcode.drop(data_PM_opcode.index)
      
#Số lượng tập train
PM = data_opcode[(data_opcode['malicious'] == True) & (data_opcode['packed'] == True)].shape[0]
PB = data_opcode[(data_opcode['malicious'] == False) & (data_opcode['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y = data_opcode['malicious'].values
data_opcode = data_opcode.drop(['malicious','packed'], axis=1)

PM = df_3000_opcode[(df_3000_opcode['malicious'] == True) & (df_3000_opcode['packed'] == True)].shape[0]
PB = df_3000_opcode[(df_3000_opcode['malicious'] == False) & (df_3000_opcode['packed'] == True)].shape[0]

df_3000_PM_opcode = df_3000_opcode[(df_3000_opcode['malicious'] == True) & (df_3000_opcode['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_opcode = df_3000_opcode.drop(df_3000_PM_opcode.index)

PM = df_3000_opcode[(df_3000_opcode['malicious'] == True) & (df_3000_opcode['packed'] == True)].shape[0]
PB = df_3000_opcode[(df_3000_opcode['malicious'] == False) & (df_3000_opcode['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2241 samples
Packed Benign: 1259 samples
Total TRAIN set: 3500 samples
Packed Malware: 1259 samples
Packed Benign: 1259 samples
Total TRAIN set: 2518 samples
Packed Malware: 1089 samples
Packed Benign: 1089 samples
Total TEST set: 2178 samples


In [30]:
# model = SelectFromModel(fsel, prefit=True)

X_train_opcode = data_opcode
X_test_opcode = df_3000_opcode.drop(['malicious','packed'], axis=1)

# data_opcode = data_opcode.loc[:, (data_opcode != 0).any(axis=0)]
x_train_opcode = data_opcode
X_test_opcode = df_3000_opcode.drop(['malicious','packed'], axis=1)
y_train_opcode = y
y_test_opcode = df_3000_opcode['malicious'].values

In [31]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(x_train_opcode, y_train_opcode)
random_forest_preds = random_forest.predict(X_test_opcode)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_opcode))

The accuracy of the Random Forests model is :	 0.529384756657484


# **rich header features**

In [32]:
themida = pd.read_pickle('/kaggle/input/packer-balance-pickle/themida_balance.pickle')
themida = themida.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
themida = themida.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = themida.select_dtypes(include=['object']).columns
themida = themida.drop(string_columns, axis=1)

In [33]:
rich = remove_cols(themida, [c for c in cols if c.startswith('ngram_')])
rich = remove_cols(themida, [c for c in cols if c.startswith('opcode_')])
rich = remove_cols(themida, [c for c in cols if c.startswith('generic_')])
rich = remove_cols(themida, [c for c in cols if c.startswith('string_')])
# rich = remove_cols(themida, [c for c in cols if c.startswith('rich_')])
rich = remove_cols(themida, [c for c in cols if c.startswith('imp_') or 'api_import_nb' == c])
rich = remove_cols(themida, [c for c in cols if c.endswith('.dll') or 'dll_import_nb' == c])
rich = remove_cols(themida, [c for c in cols if c.startswith('pesection')])
rich = remove_cols(themida, [c for c in cols if c.startswith('header_')])

removing 13000 features
removing 2500 features
removing 2 features
removing 16900 features
removing 19169 features
removing 4306 features
removing 550 features
removing 29 features


In [34]:
df_7000_rich = rich.sample(n=7000, random_state=102)
df_3000_rich = rich.drop(df_7000.index)
data_rich = df_7000_rich.sample(n=3500, random_state=102)

In [35]:
#Kiểm tra số lượng mẫu
PM = data_rich[(data_rich['malicious'] == True) & (data_rich['packed'] == True)].shape[0]
PB = data_rich[(data_rich['malicious'] == False) & (data_rich['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_rich = data_rich[(data_rich['malicious'] == True) & (data_rich['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_rich = data_rich.drop(data_PM_rich.index)
      
#Số lượng tập train
PM = data_rich[(data_rich['malicious'] == True) & (data_rich['packed'] == True)].shape[0]
PB = data_rich[(data_rich['malicious'] == False) & (data_rich['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y = data_rich['malicious'].values
data_rich= data_rich.drop(['malicious','packed'], axis=1)

PM = df_3000_rich[(df_3000_rich['malicious'] == True) & (df_3000_rich['packed'] == True)].shape[0]
PB = df_3000_rich[(df_3000_rich['malicious'] == False) & (df_3000_rich['packed'] == True)].shape[0]

df_3000_PM_rich = df_3000_rich[(df_3000_rich['malicious'] == True) & (df_3000_rich['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_rich = df_3000_rich.drop(df_3000_PM_rich.index)

PM = df_3000_rich[(df_3000_rich['malicious'] == True) & (df_3000_rich['packed'] == True)].shape[0]
PB = df_3000_rich[(df_3000_rich['malicious'] == False) & (df_3000_rich['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2241 samples
Packed Benign: 1259 samples
Total TRAIN set: 3500 samples
Packed Malware: 1259 samples
Packed Benign: 1259 samples
Total TRAIN set: 2518 samples
Packed Malware: 1089 samples
Packed Benign: 1089 samples
Total TEST set: 2178 samples


In [36]:
# model = SelectFromModel(fsel, prefit=True)

X_train_rich = data_rich
X_test_rich = df_3000_rich.drop(['malicious','packed'], axis=1)

data_rich = data_rich.loc[:, (data_rich != 0).any(axis=0)]
x_train_rich = data_rich
X_test_rich = df_3000_rich.drop(['malicious','packed'], axis=1)
y_train_rich = y
y_test_rich = df_3000_rich['malicious'].values

In [37]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train_rich, y_train_rich)
random_forest_preds = random_forest.predict(X_test_rich)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_rich))

The accuracy of the Random Forests model is :	 0.8797061524334252


# **string features**

In [38]:
themida = pd.read_pickle('/kaggle/input/packer-balance-pickle/themida_balance.pickle')
themida = themida.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
themida = themida.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = themida.select_dtypes(include=['object']).columns
themida = themida.drop(string_columns, axis=1)

In [39]:
string = remove_cols(themida, [c for c in cols if c.startswith('ngram_')])
string = remove_cols(themida, [c for c in cols if c.startswith('opcode_')])
string = remove_cols(themida, [c for c in cols if c.startswith('generic_')])
# string = remove_cols(themida, [c for c in cols if c.startswith('string_')])
string = remove_cols(themida, [c for c in cols if c.startswith('rich_')])
string = remove_cols(themida, [c for c in cols if c.startswith('imp_') or 'api_import_nb' == c])
string = remove_cols(themida, [c for c in cols if c.endswith('.dll') or 'dll_import_nb' == c])
string = remove_cols(themida, [c for c in cols if c.startswith('pesection')])
string = remove_cols(themida, [c for c in cols if c.startswith('header_')])

removing 13000 features
removing 2500 features
removing 2 features
removing 66 features
removing 19169 features
removing 4306 features
removing 550 features
removing 29 features


In [40]:
df_7000_string = string.sample(n=7000, random_state=102)
df_3000_string = string.drop(df_7000.index)
data_string = df_7000_string.sample(n=3500, random_state=102)

In [41]:
#Kiểm tra số lượng mẫu
PM = data_string[(data_string['malicious'] == True) & (data_string['packed'] == True)].shape[0]
PB = data_string[(data_string['malicious'] == False) & (data_string['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_string = data_string[(data_string['malicious'] == True) & (data_string['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_string = data_string.drop(data_PM_string.index)
      
#Số lượng tập train
PM = data_string[(data_string['malicious'] == True) & (data_string['packed'] == True)].shape[0]
PB = data_string[(data_string['malicious'] == False) & (data_string['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y = data_string['malicious'].values
data_string = data_string.drop(['malicious','packed'], axis=1)

PM = df_3000_string[(df_3000_string['malicious'] == True) & (df_3000_string['packed'] == True)].shape[0]
PB = df_3000_string[(df_3000_string['malicious'] == False) & (df_3000_string['packed'] == True)].shape[0]

df_3000_PM_string = df_3000_string[(df_3000_string['malicious'] == True) & (df_3000_string['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_string = df_3000_string.drop(df_3000_PM_string.index)

PM = df_3000_string[(df_3000_string['malicious'] == True) & (df_3000_string['packed'] == True)].shape[0]
PB = df_3000_string[(df_3000_string['malicious'] == False) & (df_3000_string['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2241 samples
Packed Benign: 1259 samples
Total TRAIN set: 3500 samples
Packed Malware: 1259 samples
Packed Benign: 1259 samples
Total TRAIN set: 2518 samples
Packed Malware: 1089 samples
Packed Benign: 1089 samples
Total TEST set: 2178 samples


In [42]:
# model = SelectFromModel(fsel, prefit=True)

X_train_string = data_string
X_test_string = df_3000_string.drop(['malicious','packed'], axis=1)

data_string = data_string.loc[:, (data_string != 0).any(axis=0)]
x_train_string = data_string
X_test_string = df_3000_string.drop(['malicious','packed'], axis=1)
y_train_string = y
y_test_string = df_3000_string['malicious'].values

In [43]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train_string, y_train_string)
random_forest_preds = random_forest.predict(X_test_string)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_string))

The accuracy of the Random Forests model is :	 0.8705234159779615


# **api features**

In [44]:
themida = pd.read_pickle('/kaggle/input/packer-balance-pickle/themida_balance.pickle')
themida = themida.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
themida = themida.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = themida.select_dtypes(include=['object']).columns
themida = themida.drop(string_columns, axis=1)

In [45]:
api = remove_cols(themida, [c for c in cols if c.startswith('ngram_')])
api = remove_cols(themida, [c for c in cols if c.startswith('opcode_')])
api = remove_cols(themida, [c for c in cols if c.startswith('generic_')])
api = remove_cols(themida, [c for c in cols if c.startswith('string_')])
api = remove_cols(themida, [c for c in cols if c.startswith('rich_')])
# api = remove_cols(themida, [c for c in cols if c.startswith('imp_') or 'api_import_nb' == c])
api = remove_cols(themida, [c for c in cols if c.endswith('.dll') or 'dll_import_nb' == c])
api = remove_cols(themida, [c for c in cols if c.startswith('pesection')])
api = remove_cols(themida, [c for c in cols if c.startswith('header_')])

removing 13000 features
removing 2500 features
removing 2 features
removing 16900 features
removing 66 features
removing 4306 features
removing 550 features
removing 29 features


In [46]:
df_7000_api = api.sample(n=7000, random_state=102)
df_3000_api = api.drop(df_7000.index)
data_api = df_7000_api.sample(n=3500, random_state=102)

In [47]:
#Kiểm tra số lượng mẫu
PM = data_api[(data_api['malicious'] == True) & (data_api['packed'] == True)].shape[0]
PB = data_api[(data_api['malicious'] == False) & (data_api['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_api = data_api[(data_api['malicious'] == True) & (data_api['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_api = data_api.drop(data_PM_api.index)
      
#Số lượng tập train
PM = data_api[(data_api['malicious'] == True) & (data_api['packed'] == True)].shape[0]
PB = data_api[(data_api['malicious'] == False) & (data_api['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y = data_api['malicious'].values
data_api = data_api.drop(['malicious','packed'], axis=1)

PM = df_3000_api[(df_3000_api['malicious'] == True) & (df_3000_api['packed'] == True)].shape[0]
PB = df_3000_api[(df_3000_api['malicious'] == False) & (df_3000_api['packed'] == True)].shape[0]

df_3000_PM_api = df_3000_api[(df_3000_api['malicious'] == True) & (df_3000_api['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_api = df_3000_api.drop(df_3000_PM_api.index)

PM = df_3000_api[(df_3000_api['malicious'] == True) & (df_3000_api['packed'] == True)].shape[0]
PB = df_3000_api[(df_3000_api['malicious'] == False) & (df_3000_api['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2241 samples
Packed Benign: 1259 samples
Total TRAIN set: 3500 samples
Packed Malware: 1259 samples
Packed Benign: 1259 samples
Total TRAIN set: 2518 samples
Packed Malware: 1089 samples
Packed Benign: 1089 samples
Total TEST set: 2178 samples


In [48]:
# model = SelectFromModel(fsel, prefit=True)

X_train_api = data_api
X_test_api = df_3000_api.drop(['malicious','packed'], axis=1)

data_api = data_api.loc[:, (data_api != 0).any(axis=0)]
x_train_api = data_api
X_test_api = df_3000_api.drop(['malicious','packed'], axis=1)
y_train_api = y
y_test_api = df_3000_api['malicious'].values

In [49]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train_api, y_train_api)
random_forest_preds = random_forest.predict(X_test_api)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_api))

The accuracy of the Random Forests model is :	 0.5298438934802571


# **dll features**

In [50]:
themida = pd.read_pickle('/kaggle/input/packer-balance-pickle/themida_balance.pickle')
themida = themida.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
themida = themida.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = themida.select_dtypes(include=['object']).columns
themida = themida.drop(string_columns, axis=1)

In [51]:
dll = remove_cols(themida, [c for c in cols if c.startswith('ngram_')])
dll = remove_cols(themida, [c for c in cols if c.startswith('opcode_')])
dll = remove_cols(themida, [c for c in cols if c.startswith('generic_')])
dll = remove_cols(themida, [c for c in cols if c.startswith('string_')])
dll = remove_cols(themida, [c for c in cols if c.startswith('rich_')])
dll = remove_cols(themida, [c for c in cols if c.startswith('imp_') or 'api_import_nb' == c])
# api = remove_cols(themida, [c for c in cols if c.endswith('.dll') or 'dll_import_nb' == c])
dll = remove_cols(themida, [c for c in cols if c.startswith('pesection')])
dll = remove_cols(themida, [c for c in cols if c.startswith('header_')])

removing 13000 features
removing 2500 features
removing 2 features
removing 16900 features
removing 66 features
removing 19169 features
removing 550 features
removing 29 features


In [52]:
df_7000_dll = dll.sample(n=7000, random_state=102)
df_3000_dll = dll.drop(df_7000.index)
data_dll = df_7000_dll.sample(n=3500, random_state=102)

In [53]:
#Kiểm tra số lượng mẫu
PM = data_dll[(data_dll['malicious'] == True) & (data_dll['packed'] == True)].shape[0]
PB = data_dll[(data_dll['malicious'] == False) & (data_dll['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_dll = data_dll[(data_dll['malicious'] == True) & (data_dll['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_dll = data_dll.drop(data_PM_dll.index)
      
#Số lượng tập train
PM = data_dll[(data_dll['malicious'] == True) & (data_dll['packed'] == True)].shape[0]
PB = data_dll[(data_dll['malicious'] == False) & (data_dll['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y = data_dll['malicious'].values
data_dll = data_dll.drop(['malicious','packed'], axis=1)

PM = df_3000_dll[(df_3000_dll['malicious'] == True) & (df_3000_dll['packed'] == True)].shape[0]
PB = df_3000_dll[(df_3000_dll['malicious'] == False) & (df_3000_dll['packed'] == True)].shape[0]

df_3000_PM_dll = df_3000_dll[(df_3000_dll['malicious'] == True) & (df_3000_dll['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_dll = df_3000_dll.drop(df_3000_PM_dll.index)

PM = df_3000_dll[(df_3000_dll['malicious'] == True) & (df_3000_dll['packed'] == True)].shape[0]
PB = df_3000_dll[(df_3000_dll['malicious'] == False) & (df_3000_dll['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2241 samples
Packed Benign: 1259 samples
Total TRAIN set: 3500 samples
Packed Malware: 1259 samples
Packed Benign: 1259 samples
Total TRAIN set: 2518 samples
Packed Malware: 1089 samples
Packed Benign: 1089 samples
Total TEST set: 2178 samples


In [54]:
# model = SelectFromModel(fsel, prefit=True)

X_train_dll = data_dll
X_test_dll = df_3000_dll.drop(['malicious','packed'], axis=1)

data_dll = data_dll.loc[:, (data_dll != 0).any(axis=0)]
x_train_dll = data_dll
X_test_dll = df_3000_dll.drop(['malicious','packed'], axis=1)
y_train_dll = y
y_test_dll = df_3000_dll['malicious'].values

In [55]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train_dll, y_train_dll)
random_forest_preds = random_forest.predict(X_test_dll)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_dll))

The accuracy of the Random Forests model is :	 0.5298438934802571


# **pe sections features**

In [56]:
themida = pd.read_pickle('/kaggle/input/packer-balance-pickle/themida_balance.pickle')
themida = themida.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
themida = themida.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = themida.select_dtypes(include=['object']).columns
themida = themida.drop(string_columns, axis=1)

In [57]:
pesection = remove_cols(themida, [c for c in cols if c.startswith('ngram_')])
pesection = remove_cols(themida, [c for c in cols if c.startswith('opcode_')])
pesection = remove_cols(themida, [c for c in cols if c.startswith('generic_')])
pesection = remove_cols(themida, [c for c in cols if c.startswith('string_')])
pesection = remove_cols(themida, [c for c in cols if c.startswith('rich_')])
pesection = remove_cols(themida, [c for c in cols if c.startswith('imp_') or 'api_import_nb' == c])
pesection = remove_cols(themida, [c for c in cols if c.endswith('.dll') or 'dll_import_nb' == c])
# pesection = remove_cols(themida, [c for c in cols if c.startswith('pesection')])
pesection = remove_cols(themida, [c for c in cols if c.startswith('header_')])

removing 13000 features
removing 2500 features
removing 2 features
removing 16900 features
removing 66 features
removing 19169 features
removing 4306 features
removing 29 features


In [58]:
df_7000_pesection = pesection.sample(n=7000, random_state=102)
df_3000_pesection = pesection.drop(df_7000.index)
data_pesection = df_7000_pesection.sample(n=3500, random_state=102)

In [59]:
#Kiểm tra số lượng mẫu
PM = data_pesection[(data_pesection['malicious'] == True) & (data_pesection['packed'] == True)].shape[0]
PB = data_pesection[(data_pesection['malicious'] == False) & (data_pesection['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_pesection = data_pesection[(data_pesection['malicious'] == True) & (data_pesection['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_pesection = data_pesection.drop(data_PM_pesection.index)
      
#Số lượng tập train
PM = data_pesection[(data_pesection['malicious'] == True) & (data_pesection['packed'] == True)].shape[0]
PB = data_pesection[(data_pesection['malicious'] == False) & (data_pesection['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y = data_pesection['malicious'].values
data_pesection = data_pesection.drop(['malicious','packed'], axis=1)

PM = df_3000_pesection[(df_3000_pesection['malicious'] == True) & (df_3000_pesection['packed'] == True)].shape[0]
PB = df_3000_pesection[(df_3000_pesection['malicious'] == False) & (df_3000_pesection['packed'] == True)].shape[0]

df_3000_PM_pesection = df_3000_pesection[(df_3000_pesection['malicious'] == True) & (df_3000_pesection['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_pesection = df_3000_pesection.drop(df_3000_PM_pesection.index)

PM = df_3000_pesection[(df_3000_pesection['malicious'] == True) & (df_3000_pesection['packed'] == True)].shape[0]
PB = df_3000_pesection[(df_3000_pesection['malicious'] == False) & (df_3000_pesection['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2241 samples
Packed Benign: 1259 samples
Total TRAIN set: 3500 samples
Packed Malware: 1259 samples
Packed Benign: 1259 samples
Total TRAIN set: 2518 samples
Packed Malware: 1089 samples
Packed Benign: 1089 samples
Total TEST set: 2178 samples


In [60]:
# model = SelectFromModel(fsel, prefit=True)

X_train_pesection = data_pesection
X_test_pesection = df_3000_pesection.drop(['malicious','packed'], axis=1)

data_pesection = data_pesection.loc[:, (data_pesection != 0).any(axis=0)]
x_train_pesection = data_pesection
X_test_pesection = df_3000_pesection.drop(['malicious','packed'], axis=1)
y_train_pesection = y
y_test_pesection = df_3000_pesection['malicious'].values

In [61]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train_pesection, y_train_pesection)
random_forest_preds = random_forest.predict(X_test_pesection)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_pesection))

The accuracy of the Random Forests model is :	 0.8888888888888888


# **pe header features**

In [62]:
themida = pd.read_pickle('/kaggle/input/packer-balance-pickle/themida_balance.pickle')
themida = themida.drop(['sample_sha1','benign','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
themida = themida.drop(['benign_vt','malicious_vt'], axis=1)
string_columns = themida.select_dtypes(include=['object']).columns
themida = themida.drop(string_columns, axis=1)

In [63]:
header = remove_cols(themida, [c for c in cols if c.startswith('ngram_')])
header = remove_cols(themida, [c for c in cols if c.startswith('opcode_')])
header = remove_cols(themida, [c for c in cols if c.startswith('generic_')])
header = remove_cols(themida, [c for c in cols if c.startswith('string_')])
header = remove_cols(themida, [c for c in cols if c.startswith('rich_')])
header = remove_cols(themida, [c for c in cols if c.startswith('imp_') or 'api_import_nb' == c])
header = remove_cols(themida, [c for c in cols if c.endswith('.dll') or 'dll_import_nb' == c])
header = remove_cols(themida, [c for c in cols if c.startswith('pesection')])
# header = remove_cols(themida, [c for c in cols if c.startswith('header_')])

removing 13000 features
removing 2500 features
removing 2 features
removing 16900 features
removing 66 features
removing 19169 features
removing 4306 features
removing 550 features


In [64]:
df_7000_header = header.sample(n=7000, random_state=102)
df_3000_header = header.drop(df_7000.index)
data_header = df_7000_header.sample(n=3500, random_state=102)

In [65]:
#Kiểm tra số lượng mẫu
PM = data_header[(data_header['malicious'] == True) & (data_header['packed'] == True)].shape[0]
PB = data_header[(data_header['malicious'] == False) & (data_header['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Chia tỉ lệ 50 - 50
data_PM_header = data_header[(data_header['malicious'] == True) & (data_header['packed'] == True)].sample(n=(PM-PB), random_state=102)
data_header = data_header.drop(data_PM_header.index)
      
#Số lượng tập train
PM = data_header[(data_header['malicious'] == True) & (data_header['packed'] == True)].shape[0]
PB = data_header[(data_header['malicious'] == False) & (data_header['packed'] == True)].shape[0]
print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TRAIN set: %i samples'% (PM+PB))

#Xử lý tập train
y = data_header['malicious'].values
data_header= data_header.drop(['malicious','packed'], axis=1)

PM = df_3000_header[(df_3000_header['malicious'] == True) & (df_3000_header['packed'] == True)].shape[0]
PB = df_3000_header[(df_3000_header['malicious'] == False) & (df_3000_header['packed'] == True)].shape[0]

df_3000_PM_header = df_3000_header[(df_3000_header['malicious'] == True) & (df_3000_header['packed'] == True)].sample(n=(PM-PB), random_state=102)
df_3000_header= df_3000_header.drop(df_3000_PM_header.index)

PM = df_3000_header[(df_3000_header['malicious'] == True) & (df_3000_header['packed'] == True)].shape[0]
PB = df_3000_header[(df_3000_header['malicious'] == False) & (df_3000_header['packed'] == True)].shape[0]

print('Packed Malware: %i samples' % PM)
print('Packed Benign: %i samples' % PB)
print('Total TEST set: %i samples'% (PM+PB))

Packed Malware: 2241 samples
Packed Benign: 1259 samples
Total TRAIN set: 3500 samples
Packed Malware: 1259 samples
Packed Benign: 1259 samples
Total TRAIN set: 2518 samples
Packed Malware: 1089 samples
Packed Benign: 1089 samples
Total TEST set: 2178 samples


In [66]:
# model = SelectFromModel(fsel, prefit=True)

X_train_header = data_header
X_test_header = df_3000_header.drop(['malicious','packed'], axis=1)

data_header = data_header.loc[:, (data_header != 0).any(axis=0)]
x_train_header = data_header
X_test_header = df_3000_header.drop(['malicious','packed'], axis=1)
y_train_header = y
y_test_header = df_3000_header['malicious'].values

In [67]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_forest = RandomForestClassifier(n_estimators=100, random_state=17)
random_forest.fit(X_train_header, y_train_header)
random_forest_preds = random_forest.predict(X_test_header)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,y_test_header))

The accuracy of the Random Forests model is :	 0.8971533516988063
